In [360]:
import numpy as np
from keras.models import Sequential, load_model, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import TensorBoard
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

from keras.applications import VGG16

import cv2

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
x = no_top_vgg16.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
predictions

<tf.Tensor 'dense_2/Sigmoid:0' shape=(?, 1) dtype=float32>

In [35]:
model = Model(no_top_vgg16.input, predictions)
model

In [46]:
# our model is a combination of all the layers in the default model and the extra layers we added
# make all the layers in the default model untrainable

for layer in no_top_vgg16.layers:
    layer.trainable = False

In [45]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='binary_crossentropy')

In [47]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
val_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train/128x128_size/subset_images/',  # this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = val_datagen.flow_from_directory(
        'data/validation/128x128_size/subset_images/',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')


model.fit_generator(
        train_generator,
        steps_per_epoch=100 // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=20 // batch_size)

Found 100 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/5
6/6 [==============================] - 14s - loss: 0.7228 - val_loss: 0.6979
Epoch 2/5
6/6 [==============================] - 11s - loss: 0.6938 - val_loss: 0.6220
Epoch 3/5
6/6 [==============================] - 13s - loss: 0.6735 - val_loss: 0.6725
Epoch 4/5
6/6 [==============================] - 11s - loss: 0.6829 - val_loss: 0.6401
Epoch 5/5
6/6 [==============================] - 13s - loss: 0.6748 - val_loss: 0.6958


In [56]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(no_top_vgg16.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 16 layers and unfreeze the rest:
for layer in model.layers[:16]:
    layer.trainable = False
for layer in model.layers[16:]:
    layer.trainable = True

0 input_2
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


In [57]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(
        train_generator,
        steps_per_epoch=100 // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=20 // batch_size)

Epoch 1/5
6/6 [==============================] - 12s - loss: 0.6832 - val_loss: 0.6385
Epoch 2/5
6/6 [==============================] - 14s - loss: 0.6486 - val_loss: 0.7000
Epoch 3/5
6/6 [==============================] - 12s - loss: 0.6598 - val_loss: 0.6715
Epoch 4/5
6/6 [==============================] - 14s - loss: 0.6561 - val_loss: 0.6933
Epoch 5/5
6/6 [==============================] - 13s - loss: 0.6587 - val_loss: 0.6794


In [59]:
model.save('VGG16_trained.h5')

## how our model looks like

In [67]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________